In [48]:
import pandas as pd
import json
import logging
from tqdm.auto import tqdm
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from openai import OpenAI
client = OpenAI()

# Ingestion

In [11]:
with open('../data/arsonor_chunks_300_50.json', 'r', encoding='utf-8') as file:
    documents = json.load(file)

In [12]:
es = Elasticsearch("http://localhost:9200")

In [13]:
index_name = "arsonor_chunks_300"

# Create index if not already created
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body={
        "mappings": {
            "properties": {
                "article_id": {"type": "keyword"},
                "title": {"type": "text"},
                "url": {"type": "keyword"},
                "category": {"type": "keyword"},
                "tags": {"type": "text"},
                "chunk_id": {"type": "keyword"},
                "chunk_text": {"type": "text"},
                # "embedding": {"type": "dense_vector", "dims": 768}  # assuming 768-dim embeddings
            }
        }
    })

In [14]:
def prepare_documents_for_indexing(docs):
    for doc in docs:
        yield {
            "_index": index_name,
            "_id": doc['chunk_id'],
            "_source": {
                "article_id": doc['article_id'],
                "title": doc['title'],
                "url": doc['url'],
                "category": doc['category'],
                "tags": doc['tags'],
                "chunk_id": doc['chunk_id'],
                "chunk_text": doc['chunk_text'],
                # "embedding": generate_embedding(doc['chunk_text'])  # Add embedding here
            }
        }

# Index the documents in bulk
bulk(es, prepare_documents_for_indexing(documents))

(572, [])

# RAG flow

In [15]:
query = 'De quel matériel ai-je besoin pour créer ma musique dans mon home-studio?'

### Two-level retrieval mechanism (article-level followed by chunk-level)

In [20]:
def elastic_search2(query, category=None, min_score=0.1):
    # First-level: Article search with diversity
    article_filter = {
        "bool": {
            "should": [
                {
                    "multi_match": {
                        "query": query,
                        "fields": ["title^3", "tags^2", "category"],
                        "type": "cross_fields",
                        "operator": "or",
                        "tie_breaker": 0.3
                    }
                }
            ],
            "filter": []
        }
    }

    if category:
        article_filter['bool']['filter'].append({"term": {"category": category}})

    article_search_body = {
        "query": article_filter,
        "size": 20,  # Increased size for more diversity
        "_source": ["article_id", "title", "category", "tags", "url"],
        "collapse": {
            "field": "article_id",  # Collapse results by article_id
            "inner_hits": {
                "name": "most_relevant_chunk",
                "size": 1,
                "sort": [{"_score": "desc"}]
            },
            "max_concurrent_group_searches": 4
        },
        "min_score": min_score
    }

    try:
        article_search_results = es.search(index=index_name, body=article_search_body)['hits']['hits']
    except Exception as e:
        print(f"Error in article search: {e}")
        return [], []

    if not article_search_results:
        return [], []

    # Extract unique article IDs
    top_article_ids = list(set(hit['_source']['article_id'] for hit in article_search_results))

    # Second-level: Diverse chunk-level search
    chunk_filter = {
        "bool": {
            "must": [
                {"terms": {"article_id": top_article_ids}},
                {
                    "multi_match": {
                        "query": query,
                        "fields": ["chunk_text^2", "title"],
                        "type": "best_fields",
                        "operator": "or",
                        "fuzziness": "AUTO"
                    }
                }
            ]
        }
    }

    chunk_search_body = {
        "query": chunk_filter,
        "size": 20,  # Increased size
        "_source": ["article_id", "chunk_id", "chunk_text", "title", "url"],
        "collapse": {
            "field": "article_id",  # Ensure chunks from different articles
            "inner_hits": {
                "name": "alternative_chunks",
                "size": 2  # Get 2 best chunks per article
            }
        },
        "min_score": min_score
    }

    try:
        chunk_search_results = es.search(index=index_name, body=chunk_search_body)['hits']['hits']
    except Exception as e:
        print(f"Error in chunk search: {e}")
        return article_search_results, []

    return article_search_results, chunk_search_results

In [21]:
def process_search_results(article_results, chunk_results):
    processed_results = []
    
    # Create a mapping of article_id to article details
    article_map = {hit['_source']['article_id']: hit['_source'] for hit in article_results}
    
    # Process chunk results and combine with article information
    for chunk_hit in chunk_results:
        article_id = chunk_hit['_source']['article_id']
        if article_id in article_map:
            article_info = article_map[article_id]
            
            # Get inner hits (alternative chunks)
            alternative_chunks = chunk_hit['inner_hits']['alternative_chunks']['hits']['hits']
            
            processed_result = {
                'article_id': article_id,
                'title': article_info['title'],
                'category': article_info.get('category', ''),
                'tags': article_info.get('tags', []),
                'url': article_info.get('url', ''),
                'chunks': [
                    {
                        'chunk_id': alt_chunk['_source']['chunk_id'],
                        'chunk_text': alt_chunk['_source']['chunk_text'],
                        'score': alt_chunk['_score']
                    }
                    for alt_chunk in alternative_chunks
                ],
                'overall_score': chunk_hit['_score']
            }
            processed_results.append(processed_result)
    
    return processed_results

In [18]:
def search_with_diversity(query, category=None):
    article_results, chunk_results = elastic_search2(query, category)
    processed_results = process_search_results(article_results, chunk_results)
    
    return processed_results

### Prompt

In [42]:
prompt_template = """
You're an audio engineer and sound designer instructor for beginners.
You're particularly specialized in audio home-studio set-up, computer music production and audio post-production in general (editing, mixing and mastering). 
Answer the QUESTION based on the CONTEXT from our arsonor knowledge database (articles).
Use only the facts from the CONTEXT when answering the QUESTION.
Finally, recommend the top 3 Arsonor articles that are the best to read for answering this question.
For each recommended article, include both its title and URL.

QUESTION: {question}

CONTEXT:
{context}

RECOMMENDED ARTICLES:
{recommendations}
""".strip()

entry_template = """
ARTICLE: {title}
KEYWORDS: {tags}
CONTENT: {chunk_text}
""".strip()

recommendation_template = """
- [{title}]({url})
""".strip()


In [46]:
def build_prompt2(query, search_results, max_context_entries=10):
    # Build context from chunks
    context_entries = []
    seen_articles = set()
    article_details = []

    # Create a flat list of all chunks with their article info
    all_chunks = []
    for result in search_results:
        article_id = result['article_id']
        
        # Store article details for recommendations
        if article_id not in seen_articles:
            seen_articles.add(article_id)
            article_details.append({
                'title': result['title'].strip(),
                'url': result.get('url', '#').strip(),
                'relevance_score': result['overall_score']
            })        

        # Add all chunks to a flat list with their article info
        for chunk in result['chunks']:
            all_chunks.append({
                'title': result['title'],
                'tags': result['tags'],
                'chunk_text': chunk['chunk_text'],
                'score': chunk['score']
            })

    # Sort all chunks by score and take top 10
    top_chunks = sorted(all_chunks, key=lambda x: x['score'], reverse=True)[:max_context_entries]
    
    # Create context entries from top chunks
    context_entries = [
        entry_template.format(
            title=chunk['title'],
            tags=chunk['tags'],
            chunk_text=chunk['chunk_text']
        )
        for chunk in top_chunks
    ]
    
    # Sort article details by relevance score and get top 3
    top_articles = sorted(article_details, key=lambda x: x['relevance_score'], reverse=True)[:3]
    
    # Create recommendations with clickable links
    recommendations = "\n".join(
        recommendation_template.format(**article) 
        for article in top_articles
    )

    # Build the full prompt
    context = "\n\n".join(context_entries)
    prompt = prompt_template.format(
        question=query,
        context=context,
        recommendations=recommendations
    )
    
    return prompt

In [47]:
def get_prompt_for_query(query, category=None, max_context_entries=10):
    search_results = search_with_diversity(query, category)
    if not search_results:
        return "No relevant articles found for your query."
    
    try:
        prompt = build_prompt2(query, search_results, max_context_entries)
        return prompt
    except Exception as e:
        logging.error(f"Error building prompt: {e}")
        return f"Error generating prompt: {str(e)}"

In [49]:
# Optional: Debug function to check context size
def debug_context_size(prompt):
    context_start = prompt.find("CONTEXT:") + 8
    context_end = prompt.find("RECOMMENDED ARTICLES:")
    context = prompt[context_start:context_end].strip()
    chunks = context.split('\n\nARTICLE:')
    return {
        'num_chunks': len(chunks),
        'total_context_length': len(context),
        'average_chunk_length': len(context) / len(chunks) if chunks else 0
    }

### Test prompt

In [63]:
prompt = get_prompt_for_query(query, category=None, max_context_entries=5)
print(prompt)

You're an audio engineer and sound designer instructor for beginners.
You're particularly specialized in audio home-studio set-up, computer music production and audio post-production in general (editing, mixing and mastering). 
Answer the QUESTION based on the CONTEXT from our arsonor knowledge database (articles).
Use only the facts from the CONTEXT when answering the QUESTION.
Finally, recommend the top 3 Arsonor articles that are the best to read for answering this question.
For each recommended article, include both its title and URL.

QUESTION: De quel matériel ai-je besoin pour créer ma musique dans mon home-studio?

CONTEXT:
ARTICLE: Comment bien débuter en MAO: le home-studio démystifié
KEYWORDS: DAW, hardware, home-studio, MAO, matériel audio, plugin, software
CONTENT: La MAO (Musique Assistée par Ordinateur) est en vogue depuis l’ère du numérique. Un ordinateur peut devenir facilement un centre de production musicale et audiovisuelle. Si en plus c’est un portable, te voilà à 

In [55]:
debug_info = debug_context_size(prompt)
print(f"Number of chunks: {debug_info['num_chunks']}")
print(f"Total context length: {debug_info['total_context_length']}")
print(f"Average chunk length: {debug_info['average_chunk_length']:.2f}")

Number of chunks: 5
Total context length: 9276
Average chunk length: 1855.20


In [56]:
def llm(prompt, model='gpt-4o-mini'):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [64]:
def rag(query, category=None, model='gpt-4o-mini'):
    prompt = get_prompt_for_query(query, category, max_context_entries=5)
    answer = llm(prompt, model=model)
    return answer

In [62]:
category = 'LE HOME STUDIO'
question = 'De quel matériel ai-je besoin pour créer ma musique dans mon home-studio?'
answer = rag(question)
print(answer)

Pour créer de la musique dans votre home-studio, vous n'avez besoin que de quelques éléments essentiels :

1. **Un ordinateur** : C'est la base de votre home-studio. Que ce soit un Mac ou un PC, un ordinateur relativement récent avec un processeur rapide et une carte mère capable d'accueillir suffisamment de RAM est recommandé. Assurez-vous également de réserver cet ordinateur exclusivement pour vos activités musicales afin d'éviter d’éventuels ralentissements.

2. **Un logiciel de production musicale (DAW)** : Ce logiciel est crucial pour composer, enregistrer, éditer, mixer et masteriser vos morceaux. Il existe de nombreux DAWs disponibles, chacun ayant ses propres fonctionnalités et interfaces.

3. **Un casque audio ou des enceintes de monitoring** : Pour écouter vos créations avec précision, un bon casque ou des enceintes de qualité est nécessaire. Cela vous permettra de faire des ajustements dans le mixage avec une meilleure référence audio.

4. **Une interface audio (optionnelle)

# Retrieval evaluation

In [148]:
df_question = pd.read_csv('../data/ground-truth-300.csv')
df_question.head()

,question,category,chunk,article
0,Quel est l'impact de l'IA sur la post-producti...,LA POST-PROD,4615db39-1,4615db39
1,Comment les outils IA simplifient-ils le trava...,LA POST-PROD,4615db39-1,4615db39
2,Quels avantages l'IA apporte-t-elle aux artist...,LA POST-PROD,4615db39-1,4615db39
3,Comment un débutant peut-il améliorer ses prod...,LA POST-PROD,4615db39-1,4615db39
4,Quelle est l'évolution des outils audio pour l...,LA POST-PROD,4615db39-1,4615db39


In [149]:
ground_truth = df_question.to_dict(orient='records')
ground_truth[0]

{'question': "Quel est l'impact de l'IA sur la post-production audio et musicale",
 'category': 'LA POST-PROD',
 'chunk': '4615db39-1',
 'article': '4615db39'}

In [22]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [24]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['chunk']
        results = search_function(q)
        relevance = [d['chunk_id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

# RAG evaluation

In [83]:
prompt2_template = """
You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()

In [84]:
len(ground_truth)

2945

In [85]:
record = ground_truth[0]
record

{'question': "Quel est l'impact de l'intelligence artificielle dans la post-production audio?",
 'category': 'LA POST-PROD',
 'chunk': '4615db39-1',
 'article': '4615db39'}

In [87]:
question = record['question']
answer_llm = rag(question)

'L\'impact de l\'intelligence artificielle (IA) dans la post-production audio est considérable et se manifeste par une amélioration des outils et processus disponibles pour les artistes, producteurs et ingénieurs du son. La démocratisation de l\'accès à des technologies avancées permet à un plus grand nombre de créateurs de produire des œuvres de qualité professionnelle sans nécessiter une formation approfondie en ingénierie du son.\n\nPremièrement, l\'IA facilite la réalisation des tâches complexes et répétitives, permettant ainsi aux artistes et producteurs de se concentrer sur les aspects créatifs de leur travail. Par exemple, des plugins d\'IA peuvent s\'occuper de tâches telles que le nettoyage et la séparation des pistes audio de manière rapide et efficace, ce qui allège la charge de travail des ingénieurs du son. Des technologies avancées comme le dé-mixage, où une piste audio est décomposée en différentes stems (voix, batterie, etc.), sont déjà courantes et en constante amélior

In [88]:
print(answer_llm)

L'impact de l'intelligence artificielle (IA) dans la post-production audio est considérable et se manifeste par une amélioration des outils et processus disponibles pour les artistes, producteurs et ingénieurs du son. La démocratisation de l'accès à des technologies avancées permet à un plus grand nombre de créateurs de produire des œuvres de qualité professionnelle sans nécessiter une formation approfondie en ingénierie du son.

Premièrement, l'IA facilite la réalisation des tâches complexes et répétitives, permettant ainsi aux artistes et producteurs de se concentrer sur les aspects créatifs de leur travail. Par exemple, des plugins d'IA peuvent s'occuper de tâches telles que le nettoyage et la séparation des pistes audio de manière rapide et efficace, ce qui allège la charge de travail des ingénieurs du son. Des technologies avancées comme le dé-mixage, où une piste audio est décomposée en différentes stems (voix, batterie, etc.), sont déjà courantes et en constante amélioration grâ

In [89]:
prompt = prompt2_template.format(question=question, answer_llm=answer_llm)
print(prompt)

You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: Quel est l'impact de l'intelligence artificielle dans la post-production audio?
Generated Answer: L'impact de l'intelligence artificielle (IA) dans la post-production audio est considérable et se manifeste par une amélioration des outils et processus disponibles pour les artistes, producteurs et ingénieurs du son. La démocratisation de l'accès à des technologies avancées permet à un plus grand nombre de créateurs de produire des œuvres de qualité professionnelle sans nécessiter une formation approfondie en ingénierie du son.

Premièrement, l'IA facilite la réalisation des tâches complexes et répétitives, permettant ainsi aux artistes et producteurs de se concentrer sur les aspects créatifs de 

In [90]:
import json

In [91]:
df_sample = df_question.sample(n=200, random_state=1)

In [92]:
sample = df_sample.to_dict(orient='records')

In [93]:
evaluations = []

for record in tqdm(sample):
    question = record['question']
    answer_llm = rag(question) 

    prompt = prompt2_template.format(
        question=question,
        answer_llm=answer_llm
    )

    evaluation = llm(prompt)
    evaluation = json.loads(evaluation)

    evaluations.append((record, answer_llm, evaluation))

  0%|          | 0/200 [00:00<?, ?it/s]

In [94]:
evaluations[0]

({'question': "Pourquoi le 32 bits float ne suffit pas avant l'enregistrement dans la DAW ?",
  'category': 'LE HOME STUDIO',
  'chunk': 'e55c4a41-6',
  'article': 'e55c4a41'},
 'Le format 32 bits float est très utile pour le traitement audio au sein d\'une station de travail audionumérique (DAW) car il offre une large plage dynamique et permet d’éviter le clipping pendant les manipulations internes. Cependant, il ne suffit pas avant l\'enregistrement dans la DAW pour plusieurs raisons.\n\nTout d\'abord, l\'avantage du 32 bits float ne s\'applique pas à la phase de l\'enregistrement, qui se déroule encore dans le domaine analogique. Pendant cette étape, il est crucial de s\'assurer que le signal audio ne dépasse pas le 0 dBFS pour éviter la saturation. De plus, une gestion rigoureuse des niveaux est essentielle dès le moment de l\'enregistrement pour maintenir une bonne qualité audio et éviter des problèmes de bruit de quantification lors des conversions numériques.\n\nEnsuite, une foi

In [95]:
df_eval = pd.DataFrame(evaluations, columns=['record', 'answer', 'evaluation'])

df_eval['id'] = df_eval.record.apply(lambda d: d['chunk'])
df_eval['question'] = df_eval.record.apply(lambda d: d['question'])

df_eval['relevance'] = df_eval.evaluation.apply(lambda d: d['Relevance'])
df_eval['explanation'] = df_eval.evaluation.apply(lambda d: d['Explanation'])

del df_eval['record']
del df_eval['evaluation']

In [96]:
df_eval.relevance.value_counts(normalize=True)

relevance
RELEVANT           0.985
PARTLY_RELEVANT    0.015
Name: proportion, dtype: float64

In [97]:
df_eval.to_csv('../data/rag-eval-gpt-4o-mini.csv', index=False)

In [99]:

df_eval[df_eval.relevance == 'PARTLY_RELEVANT']

,answer,id,question,relevance,explanation
18,Pour changer le pitch naturellement dans Ablet...,173567a9-2,Quelle méthode peut-on utiliser pour changer l...,PARTLY_RELEVANT,The generated answer discusses changing pitch ...
31,Pour éviter les pièges du make-up gain lors de...,584d0437-3,Quels conseils professionnels peut on suivre p...,PARTLY_RELEVANT,The generated answer discusses make-up gain an...
146,La question concernant l'activation du mode Th...,173567a9-11,Quelles sont les étapes pour activer le mode T...,PARTLY_RELEVANT,The generated answer acknowledges the question...
